In [1]:
%store -r model_a_s3_path
%store -r model_b_s3_path

In [2]:
%store -r s3_bucket
%store -r prefix

In [3]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

In [4]:
from sagemaker.image_uris import retrieve

image_uri = retrieve(
    "xgboost", 
    region="us-east-1", 
    version="0.90-2"
)

image_uri

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [5]:
models_path = f"s3://{s3_bucket}/model-artifacts/"

In [6]:
from sagemaker.multidatamodel import MultiDataModel

multi_model = MultiDataModel(
    name="chapter09-multi",
    model_data_prefix=models_path, 
    image_uri=image_uri,
    role=role
)

In [7]:
multi_model.add_model(model_a_s3_path)
multi_model.add_model(model_b_s3_path)

's3://sagemaker-cookbook-bucket/model-artifacts/chapter09/files/model.b.tar.gz'

In [8]:
list(multi_model.list_models())

['chapter09/files/model.a.tar.gz', 'chapter09/files/model.b.tar.gz']

In [9]:
model_a, model_b = list(
    multi_model.list_models()
)

print(model_a)
print(model_b)

chapter09/files/model.a.tar.gz
chapter09/files/model.b.tar.gz


In [10]:
%%time

endpoint_name = "chapter09-mma"

multi_model.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium', 
    endpoint_name=endpoint_name
)

Using already existing model: chapter09-multi


---------------------!CPU times: user 312 ms, sys: 32.5 ms, total: 344 ms
Wall time: 10min 32s


In [11]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = Predictor(
    endpoint_name=endpoint_name
)

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [12]:
a, b = list(multi_model.list_models())

In [16]:
a

'chapter09/files/model.a.tar.gz'

In [17]:
b

'chapter09/files/model.b.tar.gz'

In [13]:
predictor.predict(data="10,-5", target_model=a)

[0.895996630191803]

In [14]:
predictor.predict(data="10,-5", target_model=b)

[0.8308258652687073]

In [15]:
predictor.delete_endpoint()